# High performance pandas

`´´py

mask = (x>.5)&(y<.5)
df[df[mask]]

´´´

under the hood

```py
tmp1 = (x > .5)
tmp2 = (y < .5)
mask = tmp1 & tmp2 

```

use pd.eval(""),df.query() -> does elementwise operation with numexpr

In [29]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100

df1, df2,df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.936993,1.793498,0.784334,1.503682,1.758114,1.428345,-0.928185,0.401680,-1.566958,0.774828,...,-2.466627,-0.614906,1.943196,-0.445201,1.656120,1.862495,1.326114,0.813415,1.230336,0.731893
1,0.928767,-1.592302,-0.455791,-0.041175,0.769101,-0.340780,0.059620,0.768400,0.801263,-1.237079,...,-0.607535,1.002263,-2.484349,0.216593,-0.227475,0.241537,-0.533472,-1.156487,0.780268,-0.108040
2,-0.672904,-0.581982,0.048610,-1.204013,-2.395123,-0.576763,-1.882702,0.619714,-0.207640,-0.220400,...,-0.925263,-0.075078,0.406470,-0.222435,1.124488,1.267803,-0.390177,0.725118,-0.371026,-1.144648
3,-1.403415,0.320987,-0.102120,-0.599917,0.566150,-0.417548,-0.124377,0.882213,-1.165917,-0.429748,...,-1.889167,0.239447,-1.047576,0.974321,-0.082260,1.546435,-0.038073,1.824179,0.328952,-0.501776
4,-0.596805,0.710827,-0.663615,-0.514964,0.194215,0.478820,0.160737,-0.707036,-2.266497,1.699933,...,-0.037489,-1.154537,0.903830,0.026304,-1.300283,-0.552542,0.047751,-1.429802,1.528344,-0.678014


In [30]:
df1.shape

(1000000, 100)

In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [32]:
%timeit df1+df2+df3+df4
%timeit pd.eval("df1+df2+df3+df4")

1.26 s ± 81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
531 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [34]:
rolls = pd.DataFrame(np.random.randint(1,6,(6,3)), columns = ["Die1","Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,3,5,4
1,3,1,2
2,3,5,4
3,4,5,3
4,2,5,5
5,5,3,1


In [35]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace= True)
rolls

,Die1,Die2,Die3,Sum
0,3,5,4,12
1,3,1,2,6
2,3,5,4,12
3,4,5,3,12
4,2,5,5,12
5,5,3,1,9


In [36]:
high = 9
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,3,5,4,12,True
1,3,1,2,6,False
2,3,5,4,12,True
3,4,5,3,12,True
4,2,5,5,12,True
5,5,3,1,9,False


In [37]:
# traditional way to filter from a dataframe
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum
0,3,5,4,12
2,3,5,4,12
3,4,5,3,12
4,2,5,5,12


# Query
- filter using query

In [38]:
rolls.query("Sum > @high & Die1 == 3")

,Die1,Die2,Die3,Sum
0,3,5,4,12
2,3,5,4,12


In [40]:
#df_os = pd.read_csv("../Project/Data/athlete_events.csv")
df_os = pd.read_csv("https://raw.githubusercontent.com/VineelaNedunuri/Databehandling-Vineela-Nedunuri/main/Project/Data/athlete_events.csv")
df_os.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [42]:
df_os[df_os["NOC"]== "SWE"].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [43]:
df_os.query("NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [46]:
%timeit df_os[df_os["NOC"]== "SWE"]
%timeit df_os.query("NOC == 'SWE'")

22.3 ms ± 427 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.1 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%timeit df_os.query("Height > 180")
%timeit df_os[df_os["Height"] > 180]

22 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
15.5 ms ± 848 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
df_os[(df_os["Sex"]=="F")&(df_os["Height"]>180)&(df_os["NOC"]=="SWE")].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [51]:
df_os.query("Sex == 'F' & Height >180 & NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [52]:
%timeit df_os.query("Sex == 'F' & Height >180 & NOC == 'SWE'")
%timeit df_os[(df_os["Sex"]=="F")&(df_os["Height"]>180)&(df_os["NOC"]=="SWE")]

24.8 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
42.3 ms ± 509 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
